In [57]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
import scipy as sp
sns.set_context('paper')
np.set_printoptions(suppress=True)
# pd.set_option('display.max_columns', None)

In [58]:
def simu_success(tag, params):
    # which task
    if tag == "Asso-CR":
        path = "../Data/simuS1_group1_design.pkl"
        nitems = 96
        test1_num = 40
    elif tag == "Pair-CR":
        path = "../Data/simuS1_group2_design.pkl"
        nitems = 176
        test1_num = 80
    elif tag == "Item-CR":
        path = "../Data/simuS1_group3_design.pkl"
        nitems = 136
        test1_num = 80

    # load stimuli
    with open(path, 'rb') as inp:
        df_study = pickle.load(inp)
        df_test = pickle.load(inp)
    # df_study = df_study.loc[df_study.session < 200]
    # df_test = df_test.loc[df_test.session < 200]

    # load semantic matrix
    s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

    # update n_item in params to fit the study
    params.update(nitems_in_accumulator = nitems)
    # print(params)

    # run CMR
    df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params, df_study, df_test ,s_mat)
    df_simu['test'] = df_test['test']
    df_simu = df_simu.merge(df_test,on=['session','test','test_itemno1','test_itemno2'])

    # get f_in
    sessions = np.unique(df_simu.session)
    tmp_corr_fin = []
    tmp_omax_fin = []

    for sess in sessions:
        df_tmp = df_study.loc[df_study.session == sess]
        tmp1 = df_tmp.study_itemno1.to_numpy()
        tmp2 = df_tmp.study_itemno2.to_numpy()
        df_tmp2 = df_test.loc[df_test.session == sess]
        tmp3 = df_tmp2.test_itemno1[df_tmp2.test_itemno1 >= 0].to_numpy()
        tmp4 = df_tmp2.test_itemno2[df_tmp2.test_itemno2 >= 0].to_numpy()
        tmp = np.concatenate((tmp1, tmp2, tmp3, tmp4))
        tmp = np.unique(tmp)  # sort

        tmp_corr = df_simu.loc[df_simu.session == sess,"correct_ans"][test1_num:]
        corrid = np.searchsorted(tmp, tmp_corr)

        corr_fin = [f_in_dif[sess][i][id] for i, id in enumerate(corrid)]
        omax_fin = [np.max(np.delete(f_in_dif[sess][i], id)) for i, id in enumerate(corrid)]

        tmp_corr_fin = tmp_corr_fin + [-1] * test1_num + corr_fin
        tmp_omax_fin = tmp_omax_fin + [-1] * test1_num + omax_fin

    df_simu['corr_fin'] = tmp_corr_fin
    df_simu['omax_fin'] = tmp_omax_fin

    # optimal threshold for test1
    csim_two = df_simu.query("test==1").groupby("correct_ans").csim.mean()
    opt_thresh = np.mean(csim_two)
    df_simu['s_resp'] = df_simu.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

    return df_simu

In [59]:
def anal_perform(df_simu):

    # get correctness
    df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans

    # recognition performance
    df_recog = df_simu.query("test==1")
    hr_far = df_recog.groupby("correct_ans")["s_resp"].mean().to_frame(name="Yes rate")
    hr = hr_far['Yes rate'][0]
    far = hr_far['Yes rate'][1]
    z_hr_far = sp.stats.norm.ppf(hr_far)
    d_prime = z_hr_far[1].item() - z_hr_far[0].item()
    print("recognition: \n", hr_far)
    print("d_prime: ", d_prime)

    # cued recall performance
    df_cr = df_simu.query("test==2")
    p_rc = df_cr.correct.mean()
    print("cued recall: \n", p_rc)

    # analyze pair
    def get_pair(df_tmp):
        df_tmp_pair = pd.pivot_table(df_tmp,index="pair_idx",columns="test",values="correct")
        df_tmp_pair.columns = ["test1","test2"]
        df_tmp_pair.reset_index(inplace=True)
        return df_tmp_pair

    df_simu_p = df_simu.query("pair_idx >= 0")
    df_pair = df_simu_p.groupby("session").apply(get_pair).reset_index()
    df_tab = pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize=False)
    print("contingency table: \n", df_tab)
    print("contingency table norm: \n", pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize='all'))

    # compute" Q
    def Yule_Q(A, B, C, D):
        return (A * D - B * C) / (A * D + B * C)
    q = Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5)  # add 0.5
    print("Q: ", q)

    return p_rc, hr, far, d_prime, q

### Study Params

In [60]:
# study params should be the same across three groups
params = cmr.make_default_params()
params.update(
    beta_enc = 0.3,
    gamma_fc = 0.2,
    gamma_cf = 0.2,
    s_fc = 0.2,
    s_cf = 0.2,
    phi_s = 1,
    phi_d = 0.6,
    d_ass = 1,
    learn_while_retrieving = False,
    var_enc = 1,
    bad_enc_ratio = 1,
)

In [61]:
params.update(
    beta_cue = 0.4,
    beta_rec = 0.1,  # beta for retrieved item
    beta_distract = 0.1,
    # beta_rec_post = 0.1,
    c_thresh = 0.01,
    c_thresh_ass = 0.5,
    kappa = 0.01,
    lamb = 0.002,
    eta = 0.002,
    omega = 3,
    alpha = 1
)
params

{'beta_enc': 0.3,
 'beta_rec': 0.1,
 'beta_cue': 0.4,
 'beta_rec_post': 0.5,
 'beta_distract': 0.1,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.2,
 's_fc': 0.2,
 'kappa': 0.01,
 'eta': 0.002,
 'omega': 3,
 'alpha': 1,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.5,
 'd_ass': 1,
 'lamb': 0.002,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': False,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.2,
 'gamma_cf': 0.2}

### Association - CR (Group1)

In [62]:
rng = np.random.default_rng()

In [63]:
tag = "Asso-CR"
df_simu_g1 = simu_success(tag, params)

CMR2 Time: 64.69128155708313


In [64]:
n_subj = 100
n_per_subj = 10
g1_sesses = rng.choice(np.arange(1000),n_subj*n_per_subj,replace=False)

In [65]:
g1_stats = []
for i in range(n_subj):
    this_sesses = g1_sesses[n_per_subj*i:n_per_subj*(i+1)]
    this_df = df_simu_g1.loc[np.isin(df_simu_g1.session, this_sesses)].copy()    
    p_rc, hr, far, d_prime, q = anal_perform(this_df)
    g1_stats.append([p_rc, hr, far, d_prime, q])

recognition: 
              Yes rate
correct_ans          
0                0.18
1                0.89
d_prime:  2.1418932078794244
cued recall: 
 0.43
contingency table: 
 test1  0.0  1.0
test2          
0.0     21   93
1.0      1   85
contingency table norm: 
 test1    0.0    1.0
test2              
0.0    0.105  0.465
1.0    0.005  0.425
Q:  0.8582259287338894
recognition: 
              Yes rate
correct_ans          
0                0.19
1                0.89
d_prime:  2.1044244150878395
cued recall: 
 0.435
contingency table: 
 test1  0.0  1.0
test2          
0.0     19   94
1.0      3   84
contingency table norm: 
 test1    0.0   1.0
test2             
0.0    0.095  0.47
1.0    0.015  0.42
Q:  0.66565579984837
recognition: 
              Yes rate
correct_ans          
0               0.125
1               0.905
d_prime:  2.460928492544136
cued recall: 
 0.445
contingency table: 
 test1  0.0  1.0
test2          
0.0     17   94
1.0      2   87
contingency table norm: 
 test1    0

In [66]:
print(np.array(g1_stats))
print(np.mean(np.array(g1_stats),axis=0))
print(sp.stats.sem(np.array(g1_stats),axis=0))

[[0.43       0.18       0.89       2.14189321 0.85822593]
 [0.435      0.19       0.89       2.10442442 0.6656558 ]
 [0.445      0.125      0.905      2.46092849 0.73267327]
 [0.385      0.11       0.855      2.28464974 0.70524725]
 [0.455      0.115      0.895      2.4539243  0.67279693]
 [0.435      0.155      0.93       2.49101306 0.92453546]
 [0.42       0.15       0.89       2.26296151 0.95150115]
 [0.385      0.205      0.87       1.95028476 0.85918513]
 [0.43       0.19       0.84       1.87235418 0.84848485]
 [0.405      0.185      0.925      2.33600483 0.76122218]
 [0.48       0.125      0.92       2.55542094 0.94592117]
 [0.43       0.195      0.88       2.03460416 0.59815734]
 [0.39       0.155      0.875      2.16557141 0.75690203]
 [0.425      0.09       0.885      2.54111389 0.66995804]
 [0.39       0.14       0.89       2.30684746 0.8325543 ]
 [0.43       0.155      0.875      2.16557141 0.61654135]
 [0.54       0.17       0.915      2.32636906 0.87847498]
 [0.385      0

### Pair - CR (Group2)

In [67]:
tag = "Pair-CR"
df_simu_g2 = simu_success(tag, params)

CMR2 Time: 148.5393102169037


In [68]:
n_subj = 100
n_per_subj = 5
g2_sesses = rng.choice(np.arange(1000),n_subj*n_per_subj,replace=False)

In [75]:
g2_stats = []
for i in range(n_subj):
    this_sesses = g2_sesses[n_per_subj*i:n_per_subj*(i+1)]
    this_df = df_simu_g2.loc[np.isin(df_simu_g2.session, this_sesses)].copy()
    try:    
        p_rc, hr, far, d_prime, q = anal_perform(this_df)
        g2_stats.append([p_rc, hr, far, d_prime, q])
    except:
        print('-----fail-----')

recognition: 
              Yes rate
correct_ans          
0                0.01
1                0.96
d_prime:  4.07703394529301
cued recall: 
 0.345
contingency table: 
 test1  0.0  1.0
test2          
0.0      7  124
1.0      1   68
contingency table norm: 
 test1    0.0   1.0
test2             
0.0    0.035  0.62
1.0    0.005  0.34
Q:  0.4668094218415418
recognition: 
              Yes rate
correct_ans          
0               0.025
1               0.980
d_prime:  4.013712895171877
cued recall: 
 0.4
contingency table: 
 test1  0.0  1.0
test2          
0.0      4  116
1.0      0   80
contingency table norm: 
 test1   0.0   1.0
test2            
0.0    0.02  0.58
1.0    0.00  0.40
Q:  0.72294887039239
recognition: 
              Yes rate
correct_ans          
0               0.015
1               0.970
d_prime:  4.050883985735812
cued recall: 
 0.405
contingency table: 
 test1  0.0  1.0
test2          
0.0      6  113
1.0      0   81
contingency table norm: 
 test1   0.0    1.0
tes

In [70]:
print(np.array(g2_stats))
print(np.nanmean(np.array(g2_stats),axis=0))
print(sp.stats.sem(np.array(g2_stats),axis=0,nan_policy='omit'))

[[0.345      0.01       0.96       4.07703395 0.46680942]
 [0.4        0.025      0.98       4.0137129  0.72294887]
 [0.405      0.015      0.97       4.05088399 0.80647911]
 [0.36       0.015      0.98       4.22383929 0.67953668]
 [0.36       0.035      0.955      3.50730838 0.84034736]
 [0.435      0.02       0.995      4.62957821 0.4       ]
 [0.39       0.01       0.98       4.38009678 0.71272727]
 [0.39       0.005      0.965      4.38773998 0.82134571]
 [0.4        0.01       0.98       4.38009678 0.72294887]
 [0.38       0.005      0.985      4.74591968 0.63013699]
 [0.385      0.01       0.965      4.13825855 0.81782643]
 [0.385      0.025      0.97       3.84075759 0.79111111]
 [0.36       0.035      0.975      3.77187466 0.73181325]
 [0.38       0.025      0.96       3.71065006 0.8355681 ]
 [0.35       0.01       0.98       4.38009678 0.66754271]
 [0.32       0.04       0.96       3.50137214 0.42058166]
 [0.385      0.03       0.985      4.05088399 0.63650075]
 [0.475      0

### Item - CR (Group3)

In [71]:
tag = "Item-CR"
df_simu_g3 = simu_success(tag, params)

CMR2 Time: 119.77538728713989


In [72]:
n_subj = 100
n_per_subj = 5
g3_sesses = rng.choice(np.arange(1000),n_subj*n_per_subj,replace=False)

In [73]:
g3_stats = []
for i in range(n_subj):
    this_sesses = g3_sesses[n_per_subj*i:n_per_subj*(i+1)]
    this_df = df_simu_g3.loc[np.isin(df_simu_g3.session, this_sesses)].copy()    
    p_rc, hr, far, d_prime, q = anal_perform(this_df)
    g3_stats.append([p_rc, hr, far, d_prime, q])

recognition: 
              Yes rate
correct_ans          
0               0.220
1               0.825
d_prime:  1.7067825052621648
cued recall: 
 0.39
contingency table: 
 test1  0.0  1.0
test2          
0.0     33   89
1.0      2   76
contingency table norm: 
 test1    0.0    1.0
test2              
0.0    0.165  0.445
1.0    0.010  0.380
Q:  0.8394042705903463
recognition: 
              Yes rate
correct_ans          
0               0.155
1               0.855
d_prime:  2.073343650901805
cued recall: 
 0.46
contingency table: 
 test1  0.0  1.0
test2          
0.0     22   86
1.0      7   85
contingency table norm: 
 test1    0.0    1.0
test2              
0.0    0.110  0.430
1.0    0.035  0.425
Q:  0.4956268221574344
recognition: 
              Yes rate
correct_ans          
0               0.135
1               0.800
d_prime:  1.9446837897725118
cued recall: 
 0.425
contingency table: 
 test1  0.0  1.0
test2          
0.0     35   80
1.0      5   80
contingency table norm: 
 test1

In [74]:
print(np.array(g3_stats))
print(np.mean(np.array(g3_stats),axis=0))
print(sp.stats.sem(np.array(g3_stats),axis=0))

[[0.39       0.22       0.825      1.70678251 0.83940427]
 [0.46       0.155      0.855      2.07334365 0.49562682]
 [0.425      0.135      0.8        1.94468379 0.73170732]
 [0.53       0.18       0.805      1.77498245 0.5613079 ]
 [0.39       0.245      0.865      1.79337138 0.86831931]
 [0.445      0.14       0.81       1.95821564 0.56410256]
 [0.4        0.2        0.805      1.7012386  0.50024635]
 [0.45       0.155      0.845      2.03044407 0.37128435]
 [0.48       0.235      0.83       1.67664431 0.22857143]
 [0.47       0.18       0.865      2.01842764 0.62497478]
 [0.425      0.22       0.81       1.65008951 0.33621392]
 [0.385      0.2        0.84       1.83607912 0.65953654]
 [0.445      0.23       0.81       1.61674314 0.44200477]
 [0.45       0.185      0.815      1.79294673 0.23838252]
 [0.43       0.195      0.8        1.7012386  0.84055841]
 [0.43       0.27       0.85       1.64924638 0.61700058]
 [0.385      0.22       0.825      1.70678251 0.69522241]
 [0.395      0